In [ ]:
!pip install pycaret
!pip install pycaret[full]
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.1 MB/s eta 0:0

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.api as tsa
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
%matplotlib inline
from matplotlib import font_manager, rc
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='NanumBarunGothic')
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pycaret.regression import *
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb #xgb.XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.dummy import DummyRegressor

#Window size 적용한 데이터 생성

In [ ]:
df = pd.read_excel("/content/찐 최종 데이터.xlsx")
df.set_index('월별', inplace=True)

def create_lagged_dataframe(df, window_size):
    new_df = df.copy()

    for i in range(1, window_size + 1):
        lagged_df = df.shift(i)
        lagged_df.columns = [col + f'_lag{i}' for col in df.columns]
        new_df = pd.concat([new_df, lagged_df], axis=1)

    new_df = new_df.dropna()

    new_df = new_df.drop(df.columns[df.columns != '총인구'], axis=1)

    return new_df

In [ ]:
#리스트형식인 df_list에 df1 ~df40까지 저장

df = pd.read_excel("/content/찐 최종 데이터.xlsx")
df.set_index('월별', inplace=True)

def create_lagged_dataframe(df, window_size):
    new_df = df.copy()

    for i in range(1, window_size + 1):
        lagged_df = df.shift(i)
        lagged_df.columns = [col + f'_lag{i}' for col in df.columns]
        new_df = pd.concat([new_df, lagged_df], axis=1)

    new_df = new_df.dropna()

    new_df = new_df.drop(df.columns[df.columns != '총인구'], axis=1)

    return new_df


lagged_dfs = []

for i in range(1, 41):
    lagged_dfs.append(create_lagged_dataframe(df, i))
    exec(f'df_{i} = lagged_dfs[{i - 1}]')

df_list = []

for i in range(1, 41):
    df_name = f'df_{i}'
    df_list.append(eval(df_name))

#최적의 window size 찾기(1~40까지 범위)


In [ ]:
model1=xgb.XGBRegressor(random_state=2018150468)
model2=AdaBoostRegressor(random_state=2018150468)
model3=CatBoostRegressor(random_state=2018150468, task_type='GPU')
model4=RandomForestRegressor(random_state=2018150468)
model5=GradientBoostingRegressor(random_state=2018150468)
model6=KNeighborsRegressor()
model7=DummyRegressor()
model8=LGBMRegressor(random_state=2018150468)
model9=ExtraTreesRegressor(random_state=2018150468)
model10=PassiveAggressiveRegressor(random_state=2018150468)
model11=Ridge(random_state=2018150468)
model12=BayesianRidge()
model13=LinearRegression()
model14=DecisionTreeRegressor(random_state=2018150468)
model15=OrthogonalMatchingPursuit()
model16=LassoLars(random_state=2018150468)
model17=ElasticNet(random_state=2018150468)
model18=Lasso(random_state=2018150468)

In [ ]:
raw_results_df = pd.DataFrame(columns=['Trial', 'RMSE'])

def train_and_evaluate_model(model, df_list):
    raw_results_df = pd.DataFrame(columns=['Trial', 'RMSE'])

    for i, df in enumerate(df_list, start=1):
        X = df.drop(columns=['총인구'])
        y = df['총인구']

        train_size = -12
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print(f'Trial {i}: RMSE: {rmse}')

        raw_results_df = raw_results_df.append({'Trial': i, 'RMSE': rmse}, ignore_index=True)

    return raw_results_df

In [ ]:
train_and_evaluate_model(model17, df_list)

Trial 1: RMSE: 1357.3469207733797
Trial 2: RMSE: 1167.485204463939
Trial 3: RMSE: 1689.3225021007727
Trial 4: RMSE: 1635.369284714236
Trial 5: RMSE: 2363.9117364677218
Trial 6: RMSE: 2133.1952820591623
Trial 7: RMSE: 1966.5615196181268
Trial 8: RMSE: 1912.6033819988256
Trial 9: RMSE: 1759.4352315213341
Trial 10: RMSE: 1717.6201564754072
Trial 11: RMSE: 1800.5283752838602
Trial 12: RMSE: 1967.6543643814489
Trial 13: RMSE: 2181.48564142719
Trial 14: RMSE: 1906.991355727813
Trial 15: RMSE: 1842.535917200022
Trial 16: RMSE: 1733.292874095698
Trial 17: RMSE: 1735.5921591990336
Trial 18: RMSE: 1723.7590547395382
Trial 19: RMSE: 1754.4595488647792
Trial 20: RMSE: 1733.2230197612073
Trial 21: RMSE: 1740.4401864616225
Trial 22: RMSE: 1723.9195748572215
Trial 23: RMSE: 1813.0617442168107
Trial 24: RMSE: 1811.782096325879
Trial 25: RMSE: 1775.7672543489846
Trial 26: RMSE: 1777.5962844710882
Trial 27: RMSE: 1795.8786695876963
Trial 28: RMSE: 1876.9054162416332
Trial 29: RMSE: 1894.434083107886
Tri

,Trial,RMSE
0,1.0,1357.346921
1,2.0,1167.485204
2,3.0,1689.322502
3,4.0,1635.369285
4,5.0,2363.911736
5,6.0,2133.195282
6,7.0,1966.561520
7,8.0,1912.603382
8,9.0,1759.435232
9,10.0,1717.620156


#PYCARET으로 12개월 RMSE 구하기

In [ ]:
def generate_pred_model(df,i, window_size, model_type):
    df_new = create_lagged_dataframe(df, window_size)
    train, test = df_new[:-12], df_new[-12:]
    reg = setup(data=train, fold_strategy='timeseries', fold=5, target = '총인구',session_id=2018150468,data_split_shuffle=False, fold_shuffle=False)
    model = create_model(model_type, cross_validation = False)
    if model_type == 'dummy':
        final_model = finalize_model(model)
    else:
        tun_model=tune_model(model,n_iter=5)
        final_model = finalize_model(tun_model)
    pred_model=predict_model(final_model, data=test)

    residuals= pred_model['prediction_label']-pred_model['총인구']
    rmse=np.sqrt(mean_squared_error(pred_model['총인구'],pred_model['prediction_label']))
    rounded_rmse = round(rmse, 1)
    mae=mean_absolute_error(pred_model['총인구'],pred_model['prediction_label'])
    rounded_mae = round(mae, 1)
    data = {
    'Date': residuals.index,
    'Residual': residuals,
    'RMSE': [rounded_rmse] * len(residuals),
    'MAE': [rounded_mae]* len(residuals)}
    globals()[f'final_df_{i}']=pd.DataFrame(data)

    return globals()[f'final_df_{i}']

In [ ]:
generate_pred_model(df,18,10,'lasso')

,Description,Value
0,Session id,2018150468
1,Target,총인구
2,Target type,Regression
3,Original data shape,"(88, 501)"
4,Transformed data shape,"(88, 501)"
5,Transformed train set shape,"(61, 501)"
6,Transformed test set shape,"(27, 501)"
7,Numeric features,500
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Test,1532.1850,3511934.2563,1874.0155,-3.5639,1.6820,3.6796


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1116.5380,1728635.8804,1314.7760,-0.9506,1.1771,1.8159
1,1778.5031,4758383.4463,2181.3719,-2.9131,1.4611,7.6588
2,1645.7287,4927944.3985,2219.8974,-0.1237,1.3044,2.8075
3,1792.5468,5038212.1209,2244.5962,-3.2069,1.7148,5.5337
4,1910.2804,5282409.5378,2298.3493,-3.6877,1.2837,2.3390
Mean,1648.7194,4347117.0768,2051.7982,-2.1764,1.3882,4.0310
Std,278.9876,1320252.1371,370.4611,1.3860,0.1868,2.2220


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 5 candidates, totalling 25 fits


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Lasso Regression,1058.5916,2044744.6836,1429.9457,-0.5411,1.0824,2.1039


,Date,Residual,RMSE,MAE
월별,,,,
2022-05-01,2022-05-01,1867.950439,1429.9,1058.6
2022-06-01,2022-06-01,-401.567383,1429.9,1058.6
2022-07-01,2022-07-01,-1227.400269,1429.9,1058.6
2022-08-01,2022-08-01,462.579865,1429.9,1058.6
2022-09-01,2022-09-01,-2267.627197,1429.9,1058.6
2022-10-01,2022-10-01,-2373.104370,1429.9,1058.6
2022-11-01,2022-11-01,338.112701,1429.9,1058.6
2022-12-01,2022-12-01,63.403320,1429.9,1058.6
2023-01-01,2023-01-01,194.208038,1429.9,1058.6
